In [24]:
!pip install -U langchain-ollama langchain-openai

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/review_dataset

w_review_train.csv  w_review_train.parquet


In [4]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [5]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [6]:
df.shape

(40000, 2)

In [7]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")


In [8]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [9]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [10]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [11]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [12]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.17 s, sys: 35.2 ms, total: 2.2 s
Wall time: 3.2 s


In [13]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [14]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.45 s, sys: 85.7 ms, total: 2.54 s
Wall time: 6.52 s


In [15]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [16]:
coffee_reviews_df.shape

(4981, 2)

In [25]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [18]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [19]:
!ollama pull gemma3:270m

In [20]:
!ollama list

NAME           ID              SIZE      MODIFIED               
gemma3:270m    e7d36fb2c3b3    291 MB    Less than a second ago    


In [21]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ ยินดีที่ได้รู้จักนะคะ 😊




In [26]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-10-19T13:44:12.861994825Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}}]}

In [28]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [29]:
llm = OllamaLLM(model="gemma3:270m")

In [30]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ ยินดีที่ได้รู้จักค่ะ 😊



In [31]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [32]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [33]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [34]:
chain = prompt | llm

In [35]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [36]:
response

'แน่นอนค่ะ! นี่คือโค้ด Python ที่สามารถหาผลรวมของตัวเลข 1 ถึง 10 ได้ โดยใช้ `math.sum()` และ `range()`:\n\n```python\ndef find_sum_of_numbers(numbers):\n    """\n    หาผลรวมของตัวเลข 1 ถึง 10\n    """\n    try:\n        total = 0\n        for number in numbers:\n            total += number\n        return total\n    except Exception as e:\n        print(f"ข้อผิดพลาด: {e}")\n        return 0  # หรือค่าที่กำหนดอื่น\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nresult = find_sum_of_numbers(numbers)\nprint(result)\n```\n\n**คำอธิบายโค้ด:**\n\n1.  **`find_sum_of_numbers(numbers)`:** ฟังก์ชันนี้รับ `numbers` เป็น iterable ของตัวเลขเป็นค่าเริ่มต้น\n2.  **`try...except`:**  ข้อผิดพลาดที่อาจเกิดขึ้น (เช่น การพิมพ์ผิดตัวเลข) จะถูกจัดการด้วย `try...except` เพื่อป้องกันไม่ให้เกิดข้อผิดพลาด\n\n**การใช้งานโค้ด:**\n\n*   **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`:**  กำหนด iterable ของตัวเลขเป็นตัวแปรที่สามารถเป็นค่าเริ่มต้นได้\n*   **`result = find_sum_of_numbers(numbers)`:**

In [37]:
from IPython.display import display, Markdown

In [38]:
display(Markdown(response))

แน่นอนค่ะ! นี่คือโค้ด Python ที่สามารถหาผลรวมของตัวเลข 1 ถึง 10 ได้ โดยใช้ `math.sum()` และ `range()`:

```python
def find_sum_of_numbers(numbers):
    """
    หาผลรวมของตัวเลข 1 ถึง 10
    """
    try:
        total = 0
        for number in numbers:
            total += number
        return total
    except Exception as e:
        print(f"ข้อผิดพลาด: {e}")
        return 0  # หรือค่าที่กำหนดอื่น

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum_of_numbers(numbers)
print(result)
```

**คำอธิบายโค้ด:**

1.  **`find_sum_of_numbers(numbers)`:** ฟังก์ชันนี้รับ `numbers` เป็น iterable ของตัวเลขเป็นค่าเริ่มต้น
2.  **`try...except`:**  ข้อผิดพลาดที่อาจเกิดขึ้น (เช่น การพิมพ์ผิดตัวเลข) จะถูกจัดการด้วย `try...except` เพื่อป้องกันไม่ให้เกิดข้อผิดพลาด

**การใช้งานโค้ด:**

*   **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`:**  กำหนด iterable ของตัวเลขเป็นตัวแปรที่สามารถเป็นค่าเริ่มต้นได้
*   **`result = find_sum_of_numbers(numbers)`:**  เรียกฟังก์ชัน `find_sum_of_numbers()` เพื่อหาผลรวมของตัวเลข
*   **`print(result)`:**  แสดงผลลัพธ์ของฟังก์ชัน
*   **`except Exception as e:`:**  หากเกิดข้อผิดพลาด (เช่น การพิมพ์ผิดตัวเลข) จะจัดการด้วย `except` เพื่อแสดงข้อผิดพลาดให้ผู้ใช้งานเข้าใจ

**ข้อดีของโค้ดนี้:**

*   **ง่ายและเข้าใจ:**  โค้ดนี้เป็นภาษาที่ใช้งานง่ายและเข้าใจได้ง่าย ทำให้ง่ายต่อการเรียนรู้และใช้งาน
*   **ประสิทธิภาพ:**  ใช้ `math.sum()` และ `range()` ทำให้โค้ดทำงานได้อย่างรวดเร็ว
*   **มีประโยชน์:**  สามารถนำไปใช้กับตัวเลขจำนวนมากได้

**ข้อควรระวัง:**

*   **ตัวแวน:**  ในโค้ดนี้ `numbers` เป็นตัวแวน (input)  เพื่อให้สามารถนำไปใช้กับ `find_sum_of_numbers()`  ได้
*   **ค่าเริ่มต้น:**  ควรให้ค่าเริ่มต้น `0` เพื่อให้การคำนวณผลรวมเป็นไปอย่างถูกต้อง
*   **ข้อผิดพลาด:**  ควรตรวจสอบข้อผิดพลาดก่อนการใช้งานเพื่อป้องกันไม่ให้เกิดข้อผิดพลาด



In [39]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [40]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [41]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [42]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [43]:
parser_chain = StrOutputParser()

In [44]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [45]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [46]:
result

'ผลลัทธ์เป็นเครื่องมือสำคัญที่ช่วยให้เราสามารถวิเคราะห์และทำความเข้าใจสถานการณ์ต่างๆ ได้อย่างมีข้อมูลและมีประสิทธิภาพ โดยมีหลักการสำคัญดังนี้:\n\n*   **การวิเคราะห์:** ผลลัทธ์ช่วยให้เราสามารถวิเคราะห์ข้อมูลจากหลายแหล่งต่างๆ เช่น ข่าวสาร, ข้อมูลจากแหล่งอื่น, การวิเคราะห์ทางเศรษฐกิจ, การวิเคราะห์ทางวิทยาศาสตร์, และการวิเคราะห์ทางสังคม เพื่อค้นหาความสัมพันธ์และอคติที่ซับซ้อน\n*   **การทำความเข้าใจ:** ผลลัทธ์ช่วยให้เราเข้าใจความสัมพันธ์ระหว่างปัจจัยต่างๆ ที่มีผลต่อสถานการณ์ต่างๆ ได้อย่างลึกซึ้ง ทำให้เราสามารถคาดการณ์และปรับเปลี่ยนกลยุทธ์ได้อย่างรวดเร็ว\n*   **การตัดสินใจ:** ผลลัทธ์ช่วยให้เราสามารถตัดสินใจได้อย่างมีเหตุผลและมีประสิทธิภาพ โดยการพิจารณาสถานการณ์ที่เปลี่ยนแปลงไป และการวิเคราะห์ปัจจัยที่เกี่ยวข้องกับแต่ละสถานการณ์\n*   **การพัฒนา:** ผลลัทธ์เป็นเครื่องมือที่ช่วยให้เราพัฒนาตนเองและสร้างความรู้ได้อย่างต่อเนื่อง โดยการเรียนรู้จากประสบการณ์และวิเคราะห์ข้อมูลใหม่ๆ\n\n'

In [47]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON โดยมี key คือ keywords เช่น {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [48]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [49]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [50]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [51]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 87.6 ms, sys: 9.1 ms, total: 96.7 ms
Wall time: 6.74 s


In [52]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": [
    "ร้านอาหารใหญ่",
    "ห้องน้ำ",
    "กาแฟ",
    "น้ำผึ้ง",
    "ไข่มันคาว",
    "แก้วน้ำผึ้ง",
    "ลาบไข่ต้ม",
    "ไข่อร่อย",
    "เบิ้ล",
    "ขนมไทย",
    "ขนมหวาน",
    "ฟรี",
    "ราคา",
    "ราคาสูง",
    "เกินไป",
    "ไม่ไหว",
    "ซื้อไม่ไหว"
  ]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "ร้อน",
    "แก้วนึง",
    "ร้อน",
    "มาวางอยู่ข้างหน้า",
    "ซากอารยธรรมลาเต้",
    "เมพพพพ",
    "ซากอารยธรรมลาเต้เมพ",
    "ซากอารยธรรมลาเต้เมพพ",
    "เมพพพพ",
    "ซากอารยธรรมลาเต้เมพ",
    "ซากอารยธรรมลาเต้เมพพพ",
    "ซากอารยธรรมลาเต้เมพพพพ",
    "ซากอารยธรรมลาเต้เมพพพพพ",
    "ซากอารยธรรมลาเต้เมพพพพพพ",
    "ซากอารยธรรมลาเต้เมพพพพพพพ",
    "ซากอารยธรรมลาเต้เมพพพพพพพพ",
    "ซากอารยธรรมลาเต้เมพพพพพพพพพ",
    "ซากอารยธ

In [53]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [54]:
!ollama list

NAME                                  ID              SIZE      MODIFIED      
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    3 seconds ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    5 minutes ago    


In [55]:
llm3 = OllamaLLM(model="scb10x/typhoon2.1-gemma3-4b:latest")

In [56]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [57]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 105 ms, sys: 10.8 ms, total: 115 ms
Wall time: 18.2 s


In [58]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text แน่นอนครับ ผมจะช่วยคุณ Extract keywords จากรีวิวที่ให้มา โดยจะทำตามขั้นตอนที่คุณระบุไว้

ขั้นตอนที่ 1: อ่านรีวิวและเข้าใจความหมาย

รีวิวนี้พูดถึงร้านอาหารที่มีขนาดใหญ่, อาหารบางอย่างอร่อย (ทอดมันหัวปลี, พะแนงห่อไข่), แต่มีราคาแพงเกินไป, และมีขนมหวานให้ฟรี

ขั้นตอนที่ 2: ระบุหัวข้อหลักที่กล่าวถึง

*   ขนาดร้าน
*   ราคาอาหาร
*   รสชาติอาหาร (ทอดมันหัวปลี, พะแนงห่อไข่)
*   ขนมหวานฟรี

ขั้นตอนที่ 3: เลือก 3 keywords ที่สำคัญที่สุด

1.  ราคาแพง
2.  รสชาติอาหาร
3.  ขนาดร้าน

ขั้นตอนที่ 4: ตอบในรูปแบบ JSON

```json
{
  "keywords": ["ราคาแพง", "รสชาติอาหาร", "ขนาดร้าน"]
}
```

หวังว่าการ Extract keywords นี้จะเป็นประโยชน์นะครับ หากคุณมีรีวิวอื่นๆ ที่ต้องการให้ผมช่วย Extract keywords อีก สามารถแจ้งได้เลยครับ

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text แน่นอนครับ ผมจะช่วยคุณดึงคำหลักจากรีวิวที่ให้มา และจัดรูปแ

In [59]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [60]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [61]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [62]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 162 ms, sys: 11.5 ms, total: 173 ms
Wall time: 15.5 s


In [63]:
results[0]['text']['keywords']

['ราคา', 'อาหาร', 'ความพึงพอใจ']

In [64]:
results[1]['text']['keywords']

['กาแฟ', 'บรรยากาศ', 'รสชาติ']

In [65]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [66]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 139 ms, sys: 16.1 ms, total: 155 ms
Wall time: 13.6 s


In [67]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคา', 'อาหาร', 'ขนาด'], ['กาแฟ', 'บรรยากาศ', 'รสชาติ'])

In [68]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [69]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [70]:
inputs = [{"review": r} for r in sample_reviews]

In [71]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 19.3 ms, sys: 3.52 ms, total: 22.8 ms
Wall time: 2.48 s


In [72]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [73]:
sample_reviews_1000 = coffee_reviews_df['column0'].head(1000).tolist()
inputs_1000 = [{"review": r} for r in sample_reviews_1000]

BATCH_SIZE = 8

results_1000 = []

In [74]:
from tqdm import tqdm

In [75]:
for i in tqdm(range(0, len(inputs_1000), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_1000[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_1000.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 125/125 [20:29<00:00,  9.84s/it]


In [76]:
for result in results_1000[0:10]:
    print(result['text']['keywords'])

ราคา, บรรยากาศ, ไข่ต้ม
กาแฟ, บรรยากาศ, รสชาติ
โดนัท, รสชาติ, ความหวาน
วาซาบิ, ปลาซาบะ, ข้าวหน้าปลา
โดนัท, Original Glazed, คิว
วังพญาไท, นรสิงห์, คาเฟ่
อาหารทะเล, ราคา, บรรยากาศ
กาแฟ, เค้ก, พนักงาน
ฮันนี่ โทสท์, ราคา, รสชาติ
คาเฟ่, วานิลลา, คาโบนาร่า


In [77]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [78]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)

In [79]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [80]:
context_extract = context_prompt | llm3 | context_parser

In [81]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [82]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [83]:
results_1000[0]['text']['keywords'].split(',')

['ราคา', ' บรรยากาศ', ' ไข่ต้ม']

In [84]:
context_inputs = []

for review_data in tqdm(results_1000, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords'].split(',')
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 1000/1000 [00:00<00:00, 270513.00it/s]


In [85]:
len(context_inputs)

3000

In [86]:
context_results = []
for i in tqdm(range(len(results_1000)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_1000[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 1000/1000 [1:14:41<00:00,  4.48s/it]


In [87]:
results_1000[:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว',
   'บรรยากาศร้านใหญ่มากกกกกกก',
   'ไข่ต้ม']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี\nอย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ\nเขียวชอุ่ม\n\nว่าแล้วไม่รอช้

In [88]:
sentiment_inputs = []

for review_data in tqdm(results_1000, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 1000/1000 [00:00<00:00, 179673.75it/s]


In [89]:
len(sentiment_inputs)

3000

In [90]:
sentiment_inputs[:6]

[{'message': 'ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว'},
 {'message': 'บรรยากาศร้านใหญ่มากกกกกกก'},
 {'message': 'ไข่ต้ม'},
 {'message': 'กาแฟที่นี่อร่อยมากกกกกกก เรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง'},
 {'message': 'บรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี อย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ เขียวชอุ่ม'},
 {'message': 'รสชาติและกลิ่นมันไม่ธรรมดาเลย หอมๆ ถือว่าเป็นมวยรุ่นเดียวกะ Starbuck ได้'}]

In [91]:
for i in tqdm(range(len(results_1000)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_1000[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 1000/1000 [44:48<00:00,  2.69s/it]


In [92]:
results_1000[0:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว',
   'บรรยากาศร้านใหญ่มากกกกกกก',
   'ไข่ต้ม'],
  'sentiment': ['negative', 'positive', 'neutral']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี\nอย่างน้อยก็เป็นสีเขียวเล

In [93]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'].split(','))},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_1000
])

In [94]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')